In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy.io import arff
import xgboost as xgb

# Scikit-learn 工具
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve, auc, precision_recall_curve

# 模型
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE

# 设置绘图风格
sns.set(style="whitegrid", palette="muted")
import warnings
warnings.filterwarnings('ignore')

# 核心配置
FILE_PATH = 'data/processed/TimeBasedFeatures-Dataset-15s-VPN.arff' # 请确认路径正确
RANDOM_STATE = 42
N_JOBS = -1

In [ ]:
def load_and_preprocess(file_path):
    # --- 1. 加载 ARFF ---
    print(f"📂 Loading data from: {file_path}")
    data, meta = arff.loadarff(file_path)
    df = pd.DataFrame(data)
    
    # 解码 byte 字符串
    str_cols = df.select_dtypes([object]).columns
    for col in str_cols:
        df[col] = df[col].str.decode('utf-8')

    # --- 2. 标签处理 ---
    target_col = df.columns[-1]
    print(f"   -> Target column identified: '{target_col}'")
    
    # 定义标签映射逻辑 (Non-VPN -> 0, VPN -> 1)
    def map_label(label):
        label_str = str(label).lower()
        if 'non-vpn' in label_str: return 0
        if 'vpn' in label_str or 'tor' in label_str: return 1
        return 0

    df['Binary_Label'] = df[target_col].apply(map_label)
    
    # 打印分布
    y = df['Binary_Label']
    print(f"   -> Class Distribution: Normal(0)={sum(y==0)}, VPN(1)={sum(y==1)}")

    # --- 3. 特征提取 ---
    cols_to_drop = [target_col, 'Binary_Label']
    X = df.drop(columns=cols_to_drop)
    
    # 清洗数值 (NaN/Inf)
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
    X = X.replace([np.inf, -np.inf], 0)
    
    return X, y, X.columns.tolist()

In [ ]:
# 1. 加载数据
X_raw, y, feat_names = load_and_preprocess(FILE_PATH)

# 2. 划分训练集/测试集 (7:3)
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)

# 3. 全局标准化 (这对 MLP 和 Stacking 至关重要)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 保持 DataFrame 格式以便后续通过列名索引 (可选，方便调试)
X_train_df = pd.DataFrame(X_train_scaled, columns=feat_names)
X_test_df = pd.DataFrame(X_test_scaled, columns=feat_names)

print("\n✅ Data Pipeline Ready.")
print(f"   Training samples: {X_train.shape[0]}")
print(f"   Testing samples:  {X_test.shape[0]}")

In [ ]:
# --- 1. 训练基准 RF ---
print("🚀 Training Baseline Random Forest...")
rf_baseline = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=N_JOBS)
rf_baseline.fit(X_train_scaled, y_train)
y_pred_rf = rf_baseline.predict(X_test_scaled)

print(f"   -> Baseline Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"   -> Baseline F1-Score: {f1_score(y_test, y_pred_rf):.4f}")

# --- 2. 可视化 ---
plt.figure(figsize=(16, 6))

# Subplot 1: 混淆矩阵
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Random Forest)')
plt.xlabel('Predicted'); plt.ylabel('Actual')

# Subplot 2: t-SNE 降维可视化 (采样 1000 个点)
print("🎨 Generating t-SNE plot...")
plt.subplot(1, 2, 2)
sample_idx = np.random.choice(len(X_train_scaled), min(1000, len(X_train_scaled)), replace=False)
X_sample = X_train_scaled[sample_idx]
y_sample = y_train.iloc[sample_idx]

tsne = TSNE(n_components=2, random_state=RANDOM_STATE, perplexity=30)
X_embedded = tsne.fit_transform(X_sample)

scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y_sample, cmap='coolwarm', alpha=0.6, s=20)
plt.legend(handles=[mpatches.Patch(color='#6788ee', label='Normal (0)'), 
                    mpatches.Patch(color='#e46c5e', label='VPN (1)')])
plt.title('t-SNE Visualization (Feature Space)')

plt.tight_layout()
plt.show()

In [ ]:
print("⚔️ Comparing Advanced Models...")

models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=N_JOBS),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE, n_jobs=N_JOBS),
    "MLP (Neural Net)": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=RANDOM_STATE)
}

results = []
plt.figure(figsize=(10, 6))

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    # 计算 ROC
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = auc(fpr, tpr)
    
    # 记录
    f1 = f1_score(y_test, y_pred)
    results.append({"Model": name, "F1": f1, "AUC": roc_auc})
    print(f"   -> {name}: F1={f1:.4f}, AUC={roc_auc:.4f}")
    
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC={roc_auc:.3f})')

# 绘图
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc="lower right")
plt.show()

# 展示榜单
pd.DataFrame(results).sort_values(by="F1", ascending=False)

In [ ]:
print("🔧 Tuning Random Forest Hyperparameters...")

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS)
grid = GridSearchCV(rf_base, param_grid, cv=3, scoring='f1', n_jobs=N_JOBS, verbose=1)
grid.fit(X_train_scaled, y_train)

best_rf = grid.best_estimator_
best_params = grid.best_params_

print(f"\n✅ Best Params: {best_params}")
print(f"   Best CV F1: {grid.best_score_:.4f}")
print(f"   Test Set F1: {f1_score(y_test, best_rf.predict(X_test_scaled)):.4f}")

In [ ]:
print("⚖️ Final Stability Check (5-Fold Stratified CV)...")

# 定义选手
model_default = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=N_JOBS)
model_tuned = best_rf # 使用上面微调出的最佳模型

# 定义裁判 (必须 Shuffle 以打破时间序列相关性)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# 比赛
scores_def = cross_val_score(model_default, X_raw, y, cv=cv, scoring='f1', n_jobs=N_JOBS) # 注意用 X_raw
scores_tun = cross_val_score(model_tuned, X_raw, y, cv=cv, scoring='f1', n_jobs=N_JOBS)

# 结果可视化
res_df = pd.DataFrame({'Default RF': scores_def, 'Tuned RF': scores_tun})

plt.figure(figsize=(8, 5))
sns.boxplot(data=res_df, palette="Set3", width=0.5)
sns.stripplot(data=res_df, color=".25", size=5)
plt.title(f'Stability Analysis (Mean F1: Tuned={scores_tun.mean():.4f})')
plt.ylabel('F1 Score')
plt.show()

In [ ]:
print("🧱 Building Final Stacking Ensemble...")

# 1. 基学习器 (利用之前的最佳参数 + 互补模型)
estimators = [
    ('rf', best_rf), # 微调后的 RF
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE, n_jobs=N_JOBS)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=RANDOM_STATE))
]

# 2. 元学习器
final_estimator = LogisticRegression()

# 3. 训练 Stacking
clf_stack = StackingClassifier(estimators=estimators, final_estimator=final_estimator, cv=5, n_jobs=N_JOBS)
clf_stack.fit(X_train_scaled, y_train)

# 4. 评估
y_pred_stack = clf_stack.predict(X_test_scaled)
stack_f1 = f1_score(y_test, y_pred_stack)

print(f"✅ Final Stacking F1-Score: {stack_f1:.4f}")

In [ ]:
print("🎯 Optimizing Decision Threshold...")

# 获取概率
y_prob_stack = clf_stack.predict_proba(X_test_scaled)[:, 1]

# 遍历阈值寻找最佳 F1
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob_stack)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
best_idx = np.argmax(f1_scores)
best_thresh = thresholds[best_idx]
best_f1_val = f1_scores[best_idx]

print(f"   -> Default Threshold (0.50) F1: {stack_f1:.4f}")
print(f"   -> Optimal Threshold ({best_thresh:.4f}) F1: {best_f1_val:.4f}")

# 可视化
plt.figure(figsize=(10, 5))
plt.plot(thresholds, f1_scores[:-1], label='F1 Score', color='navy')
plt.axvline(best_thresh, color='red', linestyle='--', label=f'Best Threshold: {best_thresh:.2f}')
plt.title('F1 Score vs Decision Threshold')
plt.xlabel('Threshold'); plt.ylabel('F1 Score')
plt.legend(); plt.show()